In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['번역(다국어-다국어)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess

from collections import defaultdict

def find_consecutive_ranges(input_data):
    ranges = defaultdict(list)
    n = len(input_data)
    
    for start in range(n):
        current_intersection = input_data[start]
        for end in range(start + 1, n + 1):
            if end > start + 1:
                current_intersection &= input_data[end - 1]
            
            if len(current_intersection) > 1:
                sorted_items = tuple(sorted(current_intersection))
                if not ranges[sorted_items] or ranges[sorted_items][-1][1] < start:
                    ranges[sorted_items].append((start, end))
                else:
                    ranges[sorted_items][-1] = (ranges[sorted_items][-1][0], end)
            
            # If the intersection is empty, no need to check further
            if not current_intersection:
                break

    return ranges
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    utt_dict = dict()
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        #### data preprocess
        for line in source_data.readlines():
            line = json.loads(line)
            script_id = line['typeInfo']['script']['scriptFileName']
            text_id = line['dialogs']['textNumber']
            language = line['typeInfo']['language']
            utt = {language: line['dialogs']['text']}
            if script_id not in utt_dict:
                utt_dict[script_id] = {text_id: utt}
            else:
                if text_id in utt_dict[script_id]:
                    utt_dict[script_id][text_id].update(utt)
                else:
                    utt_dict[script_id][text_id] = utt
        #### data preprocess end     
            
    for key, lang_data in tqdm(utt_dict.items(), desc=split):
        lang_data = sorted(lang_data.items())
        lang_key_data = [set(v[1].keys()) for v in lang_data]
        output = find_consecutive_ranges(lang_key_data)
        for items, spans in output.items():
            for span in spans:
                span_data = lang_data[span[0]:span[1]]
                
                #### 번역(다국어-다국어)
                data = {'en': None, 'ko': None,
                        'jp': None, 'es' : None}
                for item in items:
                    data[item] = ' '.join([d[1][item] for d in span_data])
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 번역(다국어-다국어) end

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 3387/3387 [00:00<00:00, 15895.25it/s]
